In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))

In [3]:
import wiki

path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [6]:
topic = 'earth science'
%time dump.load_page('Index of {} articles'.format(topic))
links = dump.links
dump.links[:5]

CPU times: user 18.8 ms, sys: 822 µs, total: 19.6 ms
Wall time: 34.1 ms


['Earth science', 'Science', 'Earth (planet)', 'Planetary science', 'Life']

#### Test network generation

In [7]:
net = wiki.Net()
net.build_graph(dump, nodes=[str(l) for l in links])

wiki.Net: traversing Wikipedia...
wiki.Net: depth = 0
wiki.Net: len(queue) = 530
wiki.Net: depth = 1
wiki.Net: removing isolates...
wiki.Net: adding years...
wiki.Net: filling empty years...


In [8]:
[len(links), len(net.graph.nodes)]

[138, 86]

In [9]:
net.save_graph('test.gexf')

In [10]:
new_net = wiki.Net()
new_net.load_graph('test.gexf')
len(new_net.graph.nodes)

86

In [11]:
[n for n in net.graph.nodes][:5]

['Planetary science',
 'Earth science',
 'Lithosphere',
 'Hydrosphere',
 'Biosphere']

In [12]:
[y for y in net.years][:5]

[-4000, -3999, -3000, -800, -799]

In [13]:
net._numbered = None
[n for n in net.numbered.nodes][:5]

[0, 1, 6, 34, 40]

#### Test clique to barcodes

In [14]:
net._cliques = None
%time net.cliques[:4]
%time net.cliques[:4]

CPU times: user 1.86 ms, sys: 64 µs, total: 1.92 ms
Wall time: 2 ms
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


[[0], [1], [6], [34]]

In [19]:
net.nodes_for_year[-3000]

[9, 12]

In [20]:
%time net.filtration

CPU times: user 30.6 ms, sys: 874 µs, total: 31.5 ms
Wall time: 30.8 ms


Filtration with 567 simplices

In [21]:
%time net.persistence

CPU times: user 350 µs, sys: 14 µs, total: 364 µs
Wall time: 366 µs


Reduced matrix with 567 columns

In [22]:
net._barcodes = None
%time net.barcodes
%time net.barcodes

wiki.Net: computing barcodes... (skip negatives)
wiki.Net: barcode 565/567
CPU times: user 157 ms, sys: 44.7 ms, total: 202 ms
Wall time: 179 ms
CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10 µs


,dim,birth,death,birth simplex,death simplex,birth nodes,death nodes
0,0,-4000,inf,[Hydrology],[],[Hydrology],[]
1,0,-3999,-3999.0,[Cryosphere],"[Hydrology, Cryosphere]",[Cryosphere],[Cryosphere]
2,0,-3000,263.0,[Physics],"[Oceanography, Hydrology]",[Physics],[Oceanography]
3,0,-3000,-3000.0,[Mathematics],"[Physics, Mathematics]",[Mathematics],"[Physics, Mathematics]"
4,0,-800,263.0,[Geography],"[Physics, Oceanography]",[Geography],[Oceanography]
5,0,-800,-799.0,[Cartography],"[Cartography, Geoinformatics]",[Cartography],[Geoinformatics]
6,0,-799,-799.0,[Geoinformatics],"[Geography, Geoinformatics]",[Geoinformatics],[Geoinformatics]
7,0,-350,-350.0,[Meteorology],"[Hydrology, Meteorology]",[Meteorology],[Meteorology]
8,0,-349,-349.0,[Hydrometeorology],"[Hydrology, Hydrometeorology]",[Hydrometeorology],[Hydrometeorology]
9,0,-300,-300.0,[Chemistry],"[Physics, Chemistry]",[Chemistry],[Chemistry]


#### Test weighting with tf-idf model

In [ ]:
import pickle
import gensim.utils as gu

path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [ ]:
net.graph['Chemistry']

In [ ]:
w_net = wiki.Net()
w_net.build_graph(dump, nodes=[str(l) for l in links], model=tfidf, dct=dct)

In [ ]:
w_net.graph['Chemistry']